# Part 2 Machine Learning
Eric Pahl

The line of questioning for this part of Eric Pahl’s comprehensive exam probes the depth of his understanding related to common assertions made about random decision forest.

## Problem 2.1: Implementation

Gini Impurity Index is a standard criterion used when constructing individual decision trees for a random forest implementation. Reusing the data set “ExampleData.csv” provided in Part 1, but now assuming that this data set represents a retrospective observational study that records which treatments a physician chooses based on the other variables.  For Part 2, interpret the variables as:

|Variable name|Data Type|Description|
|-------------|---------|-----------|
|Treatment<br>(output category)|Categorical|A: Branded drug <br>B: Generic Drug|
|Age|Integer|Age in years after birth|
|Sex|Categorical|M: Male,<br> F: Female|
|OverallHealthIdx|Integer|Score from a self-reported questionnaire|
|Time|Integer|Number of hours since emergency room admission|
|Censored <br>(poorly named variable)|Categorical|The identifier of the physician type:<br>0: Internal Medicine doctor, <br>1: Family Medicine doctor|

### a) Create a fork of the code associated with the article (https://github.com/joachimvalente/decision-tree-cart) for the modifications.  Store the results of part1 under your forked repositories branch named “part1_comps”.
Done. See https://github.com/ericpahl/decision-tree-cart/tree/part1_comps.

### b)	Modify the source code to generate a pictorial representation of the tree associated with applying the described algorithm to the data provided using a maximum tree depth of 1,000,000.
Carefully report the data manipulations or code modifications that are necessary to complete this task.  It is *NOT* acceptable to pre-process or modify the input data outside of this codebase.  The resulting changes needed to investigate the newly provided data set should not break the existing functionality of the program.

1. I read the Cart.py documentation and ran the main function to confirm it was working in its default configuration with prespecified datasets (iris, wifi, and breast).
1. I noticed that ExampleData.csv included incompatible datatypes like char and str when only numeric data were acceptable for the DecisionTreeClassifier.
1. I created two checks and conversions in the fit function within DecisionTreeClassifier code cart.py. 
    1. Convert the outcome varible to numeric 0 to n-1
    1. Convert binary str variables to 0 and 1
1. I ran the main function again to make sure that I did not break any existing functionality required for iris, breast, and wifi.
1. I fit the ExampleData successfully with max_depth 1,000,000 and used the debug function to print the tree.
 

In [1]:
#%pip3 install pandas
import numpy as np
import pandas as pd
import cart
import categorical_gini
import continuous_gini
import minimal_cart
import tree

df = pd.read_csv('~/ExampleData.csv')
df.head()

,Treatment,Age,Sex,OverallHealthIdx,Time,Censored
0,A,95,M,56,39,1
1,A,85,M,66,37,1
2,A,73,F,67,39,1
3,A,62,M,39,23,0
4,A,62,M,92,29,0


In [2]:
df.describe(include='all')

,Treatment,Age,Sex,OverallHealthIdx,Time,Censored
count,200,200.000000,200,200.000000,200.000000,200.000000
unique,2,NaN,2,NaN,NaN,NaN
top,B,NaN,M,NaN,NaN,NaN
freq,100,NaN,105,NaN,NaN,NaN
mean,NaN,77.200000,NaN,67.220000,32.245000,0.360000
std,NaN,11.285692,NaN,15.213113,9.092212,0.481205
min,NaN,48.000000,NaN,35.000000,14.000000,0.000000
25%,NaN,70.000000,NaN,55.750000,25.000000,0.000000
50%,NaN,78.000000,NaN,67.000000,31.000000,0.000000
75%,NaN,85.000000,NaN,76.000000,39.000000,1.000000


In [3]:
# convert to numpy array
arr = df.to_numpy()

In [4]:
# predictors
X = arr[:,1:]

In [5]:
# outcome
y = arr[:,0]

In [6]:
# create the classifier per max_depth
clf = cart.DecisionTreeClassifier(max_depth=1000000)

In [7]:
# train the classifier with predictors and outcome
clf.fit(X, y)

In [8]:
# print the tree
clf.debug(list(df.columns[1:]),
          list(list(np.unique(df['Treatment']))),
          False)

### c)	In your report, describe the Gini impurity Index, including its origins, intuitive meaning, and mathematical implementation.
There are two common splitting criteria for nodes when training decision-tree-based classifiers, Gini Index and Information Gain. The Gini Impurity Index (gini) is a simple measure of impurity in a set of observations. The gini was introduced in the early 20th century by Corrado Gini. 

Intuitively, gini is the chance of incorrectly classifying any given observations in the set. If there is a 50/50 split, then the gini is $0.5$. If there is a 75/25 split, then the gini is calculated by adding the incorrect probabilites, $0.75 * 0.25 + 0.25 * 0.75 = 0.375$ the impurity is reduced when the set becomes more homogeneous. The equation is $gini = \sum^C_{i=1} p(i) * (1-p(i))$ where $C$ is number of classes and $p(i)$ is the probability of picking an observation with class $i$.


### d)	Provide a simple function implementing the mathematical formulation of the Gini impurity Index that computes the Gini impurity index
In the Python programming language, in a new file named "categorical_gini.py" under the “part1_comps” branch.  Report the Gini impurity index for the categorical variables ["Sex," "Censored"] with respect to Treatment variable in the table above. The resulting code must be well documented and easy to understand by other non-expert programmers. Report any assumptions or limitations to apply this function.  

For the categorical_gini.py function, there are some assumptions and limitations. This function is only appropriate for binary categorical variables and categorical outcomes. The function assumes that there will be three inputs, an __X__ numpy array of observations (row) and variables (columns), __y__ vector of outcomes, and __varIdx__ the column index of __X__ of the variable in question to calculate the gini index. 




In [9]:
import categorical_gini as cat_gini

# index of variable to calculate
sexIdx = 1       # idx is 1 for Sex
censoredIdx = 4  # idx is 4 for Censored

print("Sex Gini: " + str(cat_gini.categorical_gini(X,y,sexIdx)))
print("Censored Gini: " + str(cat_gini.categorical_gini(X,y,censoredIdx)))

Sex Gini: 0.4999496234320136
Censored Gini: 0.48755711684992287


### e)	Provide a simple function implementing the mathematical formulation for identifying both the optimal Gini impurity index and the threshold for optimality of a continuous variable.
In the Python programming language, in a new file named "continuous_gini.py" under the “part1_comps” branch.  Report both the optimal Gini impurity index and the associated threshold for the categorical variables ["Age," "OverallHealthIdx," "Time"] with respect to Treatment in the table above. The resulting code must be well documented and easy to understand by other non-expert programmers. Report any assumptions or limitations to apply this function.

For the continuous_gini.py function, there are some assumptions and limitations. This function is only appropriate for continuous variables and categorical outcomes. The function assumes that there will be three inputs, an __X__ numpy array of observations (row) and variables (columns), __y__ vector of outcomes, and __varIdx__ the column index of __X__ of the variable in question to calculate the gini index. 


In [10]:
import continuous_gini as cont_gini
# index of variable to calculate
TimeIdx = 3       # idx is 3 for Time
AgeIdx = 0       # idx is 0 for Age
OverallHealthIdx = 2 # idx is 2 for overallhealthidx

print("Age (Gini,Threshold): " + str(cont_gini.continuous_gini(X,y,AgeIdx)))
print("OverallHealthIdx (Gini,Threshold): " + str(cont_gini.continuous_gini(X,y,OverallHealthIdx)))
print("Time (Gini,Threshold): " + str(cont_gini.continuous_gini(X,y,TimeIdx)))


Age (Gini,Threshold): (0.4892307692307692, 82.5)
OverallHealthIdx (Gini,Threshold): (0.4899218428630194, 90.5)
Time (Gini,Threshold): (0.4695495495495496, 46.5)
